In [114]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

from baseline import Baseline
from system_t import System_T

from data import load_data
from DatasetExt import *
from Misc import *




# Load AdultCensus Dataset

In [115]:
dataset_name='mobility'
datapool, test_orig_df=load_data(dataset_name)
Target_attribute, Sensitive_attribute, protected_group, privileged_group= get_target_sensitive_attribute(dataset_name)
dataset_orig_df = pd.concat([datapool, test_orig_df], ignore_index=True)
train_pct=5
train_size=int(dataset_orig_df.shape[0]*(train_pct/100))
train_smart_df= smart_sample(datapool, train_size, 50, 90, dataset_name)  #keep it balance for the sake of accuracy
datapool = datapool.drop(train_smart_df.index)
X_train = train_smart_df.drop(columns=[Target_attribute])
y_train = train_smart_df[Target_attribute]
X_test = test_orig_df.drop(columns=[Target_attribute])
y_test = test_orig_df[Target_attribute]
X_datapool = datapool.drop(columns=[Target_attribute])
y_datapool = datapool[Target_attribute]

#x_data = pd.concat([x_train, x_test])
#y_data = pd.concat([y_train, y_test])
#x_data = x_data.copy()
X_train["sex_Female"] = (X_train["RAC1P"] == 0).astype(int)
X_train["sex_Male"] = (X_train["RAC1P"] == 1).astype(int)
X_datapool["sex_Female"] = (X_datapool["RAC1P"] == 0).astype(int)
X_datapool["sex_Male"]   = (X_datapool["RAC1P"] == 1).astype(int)
#X_datapool["race_White"] = (X_datapool["race"]   == 1).astype(int)
#X_datapool["race_Black"] = (X_datapool["race"]   == 0).astype(int)

# Now use the one-hot approach above to build slice_index


In [116]:
X_datapool

,AGEP,SCHL,MAR,SEX,DIS,ESP,CIT,MIL,ANC,NATIVITY,...,DREM,RAC1P,GCL,COW,ESR,WKHP,JWMNP,PINCP,sex_Female,sex_Male
32344,32.0,18.0,1.0,1,1.0,0.0,1.0,4.0,1.0,1.0,...,1.0,1,2.0,6.0,1.0,4.0,55.0,-1000.0,0,1
77114,26.0,18.0,5.0,1,2.0,0.0,1.0,4.0,2.0,1.0,...,2.0,1,0.0,6.0,1.0,40.0,60.0,28000.0,0,1
2623,23.0,19.0,5.0,1,2.0,0.0,1.0,3.0,2.0,1.0,...,2.0,1,0.0,5.0,4.0,45.0,5.0,22700.0,0,1
59590,24.0,16.0,5.0,1,2.0,0.0,1.0,4.0,1.0,1.0,...,2.0,1,0.0,0.0,6.0,0.0,0.0,0.0,0,1
14858,28.0,21.0,5.0,1,2.0,0.0,1.0,4.0,2.0,1.0,...,2.0,1,0.0,1.0,1.0,60.0,45.0,138470.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71967,33.0,21.0,5.0,1,2.0,0.0,4.0,4.0,1.0,2.0,...,2.0,1,2.0,1.0,1.0,40.0,5.0,300000.0,0,1
28401,24.0,21.0,1.0,0,2.0,0.0,5.0,4.0,1.0,2.0,...,2.0,1,0.0,3.0,6.0,15.0,0.0,18000.0,0,1
75640,26.0,19.0,5.0,1,2.0,0.0,1.0,4.0,4.0,1.0,...,2.0,1,0.0,7.0,1.0,35.0,3.0,102000.0,0,1
48707,28.0,19.0,1.0,1,2.0,0.0,1.0,4.0,1.0,1.0,...,2.0,1,0.0,1.0,1.0,40.0,5.0,42000.0,0,1


# Basic setting: slices have the same amounts of data

In [117]:
def shuffle(data, label):
    shuffle = np.arange(len(data))
    np.random.shuffle(shuffle)
    data = data[shuffle]
    label = label[shuffle]
    return data, label

initial_data_array = []
val_data_dict = []
add_data_dict = []
    
val_data_num = 500

feature_index = 0
gender_list = ["sex_Female", "sex_Male"]
race_list = ["race_White", "race_Black"]

for gender in gender_list:
    #for race in race_list:
        if feature_index < 2:
            data_num = 200
        else:
            data_num = 400
        initial_data_array.append(data_num)
        #temp_x, temp_y = x_data[(x_data[gender] == 1) & (x_data[race] == 1)], y_data[(x_data[gender] == 1) & (x_data[race] == 1)]
        temp_x, temp_y = X_datapool[(X_datapool[gender] == 1)], y_datapool[(X_datapool[gender] == 1)]
        xtrain_temp, ytrain_temp = X_train[(X_train[gender] == 1)], y_train[(X_train[gender] == 1)]

        val_data_dict.append((temp_x[data_num:data_num+val_data_num].to_numpy(), tf.keras.utils.to_categorical(temp_y[data_num:data_num+val_data_num])))
        add_data_dict.append((temp_x[data_num+val_data_num:].to_numpy(), tf.keras.utils.to_categorical(temp_y[data_num+val_data_num:])))
        
        if feature_index == 0:
            train_data = xtrain_temp.copy()
            train_label = ytrain_temp.copy()
            val_data = temp_x[data_num:data_num+val_data_num]
            val_label = temp_y[data_num:data_num+val_data_num]
        else:
            train_data = pd.concat([train_data, xtrain_temp.copy()])
            train_label = pd.concat([train_label, ytrain_temp.copy()]) 
            val_data = pd.concat((val_data, temp_x[data_num:data_num+val_data_num]))
            val_label = pd.concat((val_label, temp_y[data_num:data_num+val_data_num])) 
        feature_index += 1
        
num_label = len(np.unique(train_label))
num_class = feature_index 
print("Number of slices : %d, %d" % (num_class, num_label))

train_data = train_data.to_numpy()
train_label = tf.keras.utils.to_categorical(train_label)

val_data = val_data.to_numpy()
val_label = tf.keras.utils.to_categorical(val_label)
print(val_label.sum(axis=0))

train_data, train_label = shuffle(train_data, train_label)

Number of slices : 2, 2
[251. 749.]


# Define slices

In [118]:
print(train_data.shape, train_label.shape)

(4016, 23) (4016, 2)


In [119]:
# feature_list must match the column order in your X (e.g., x_data.columns)
feature_list = np.array(X_datapool.columns)

# If you already have these lists, keep them (ensure this order):
gender_list = ["sex_Female", "sex_Male"]
#race_list   = ["race_White", "race_Black"]

# Build slice_index and slice_desc in the SAME order: gender major, race minor
slice_index = []
slice_desc  = []
for g in gender_list:
    #for r in race_list:
        ind_g = int(np.where(feature_list == g)[0][0])
        #ind_r = int(np.where(feature_list == r)[0][0])
        slice_index.append(ind_g)

        # Pretty description that matches this exact ordering
        g_name = "Female" if g.endswith("Female") else "Male"
        #r_name = r.split("_", 1)[1].title()  # "race_White" -> "White"
        slice_desc.append(f"{g_name}")

# OPTIONAL: print initial sizes aligned with the true order
for i, desc in enumerate(slice_desc):
    print(f"Slice: {desc}, Initial size: {initial_data_array[i]}")

# Derive priv/prot slice indices programmatically (future-proof if you add more races)
male_col   = int(np.where(feature_list == "sex_Male")[0][0])
female_col = int(np.where(feature_list == "sex_Female")[0][0])

print(slice_index)

priv_slices = [i for i, a in enumerate(slice_index) if a == male_col]     # Male-*
prot_slices = [i for i, a in enumerate(slice_index) if a == female_col] # Female-*

print("Privileged (Male-*) slice indices:", priv_slices)
print("Protected  (Female-*) slice indices:", prot_slices)


Slice: Female, Initial size: 200
Slice: Male, Initial size: 200
[21, 22]
Privileged (Male-*) slice indices: [1]
Protected  (Female-*) slice indices: [0]


In [120]:
slice_index

[21, 22]

In [121]:
cost_func = [1] * num_class
lr = 0.001

ori = Baseline(
    (train_data, train_label),
    (val_data, val_label),
    val_data_dict,
    initial_data_array,
    num_class, num_label,
    slice_index, add_data_dict,
    method='Uniform',
    favorable_label=1,                    # >50K
    privileged_slice_indices=priv_slices,
    protected_slice_indices=prot_slices
)
ori.performance(budget=0, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)


Method: Uniform, Budget: 0
======= Collect Data =======
[0 0]
======= Performance (Group Disparity: prot - priv) =======
Loss: 0.76251 (0.05932)
Group Disparity (prot - priv): -0.34192 (0.08615)
Priv positive rate: 0.96826 (0.02374)
Prot positive rate: 0.62635 (0.09559)



In [122]:
'''budget = 5000
cost_func = [1] * num_class
lr = 0.001

uni = Baseline(
    (train_data, train_label),
    (val_data, val_label),
    val_data_dict,
    initial_data_array,
    num_class, num_label,
    slice_index, add_data_dict,
    method='Uniform',
    favorable_label=1,                    # <=50K
    privileged_slice_indices=priv_slices,
    protected_slice_indices=prot_slices
)
uni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)'''


"budget = 5000\ncost_func = [1] * num_class\nlr = 0.001\n\nuni = Baseline(\n    (train_data, train_label),\n    (val_data, val_label),\n    val_data_dict,\n    initial_data_array,\n    num_class, num_label,\n    slice_index, add_data_dict,\n    method='Uniform',\n    favorable_label=1,                    # <=50K\n    privileged_slice_indices=priv_slices,\n    protected_slice_indices=prot_slices\n)\nuni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)"

In [123]:
'''budget = 5000
cost_func = [1] * num_class
lr = 0.001

uni = Baseline(
    (train_data, train_label),
    (val_data, val_label),
    val_data_dict,
    initial_data_array,
    num_class, num_label,
    slice_index, add_data_dict,
    method='Waterfilling',
    favorable_label=1,                    # <=50K
    privileged_slice_indices=priv_slices,
    protected_slice_indices=prot_slices
)
uni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)'''

"budget = 5000\ncost_func = [1] * num_class\nlr = 0.001\n\nuni = Baseline(\n    (train_data, train_label),\n    (val_data, val_label),\n    val_data_dict,\n    initial_data_array,\n    num_class, num_label,\n    slice_index, add_data_dict,\n    method='Waterfilling',\n    favorable_label=1,                    # <=50K\n    privileged_slice_indices=priv_slices,\n    protected_slice_indices=prot_slices\n)\nuni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)"

In [124]:
cost_func = [1] * num_class
lr = 0.001

In [125]:
import copy
import numpy as np

fairness = []   # SP gap (protected - privileged)
accuracy = []   # overall accuracy on val
budgets  = []

for i in range(0, 11):
    budget = 2 * i * X_datapool.shape[0] // 100  # 0%, 2%, ..., 20% of pool size
    method = 'Moderate'
    budgets.append(budget)

    # Fresh System_T each sweep (it deep-copies inputs internally)
    st = System_T(
        (train_data, train_label),
        (val_data, val_label),
        val_data_dict,
        initial_data_array,
        num_class, num_label,
        slice_index,
        add_data_dict,
        favorable_label=1,
        privileged_slice_indices=priv_slices,
        protected_slice_indices=prot_slices
    )

    # Run Slice Tuner (collection). We don’t rely on its return value here.
    _ = st.selective_collect(
        budget=budget,
        k=10,
        batch_size=32,
        lr=lr,
        epochs=2000,
        cost_func=cost_func,
        Lambda=0.1,
        num_iter=5,
        slice_desc=slice_desc,
        strategy=method,
        show_figure=False,
        # choose external model used for reporting if you want it printed during run:
        ext_model_type="logreg",  # or "svm"
        ext_C=1.0,
        ext_kernel="rbf"
    )

    # Programmatic metrics: train external model on the collected train and evaluate on val
    acc, sp_gap = st.evaluate_sklearn(model_type="logreg", C=1.0, kernel="rbf")
    accuracy.append(acc)
    fairness.append(sp_gap)

# Report
for b, acc, sp in zip(budgets, accuracy, fairness):
    print(f"Budget={b:6d} | Accuracy={acc:.4f} | SP (prot-priv)={sp:.5f}")

# If you also want arrays for plotting:
fairness = np.array(fairness)
accuracy = np.array(accuracy)
budgets  = np.array(budgets)



======= Collect Data =======
[0 0]
Total Cost: 1, Remaining Budget: -1.0

======= Performance (FC summary) =======
[0 0]
Number of iteration: 1
Strategy: Moderate, Lambda: 0.1, Initial Budget: 0
FC Loss: 0.83178 (0.24322)
FC Group Disparity (prot - priv): -0.31297 (0.04155)
FC Priv positive rate: 0.90858 (0.10440)
FC Prot positive rate: 0.59561 (0.12140)


======= External Model Evaluation =======
Model: logreg (C=1.0)
Accuracy (val): 0.7520
Statistical Parity (prot - priv): -0.06587
======= Collect Data =======
[719 485]
Total Cost: 1205, Remaining Budget: -1.0

======= Performance (FC summary) =======
[719 485]
Number of iteration: 1
Strategy: Moderate, Lambda: 0.1, Initial Budget: 1204
FC Loss: 0.78037 (0.15315)
FC Group Disparity (prot - priv): -0.26747 (0.02000)
FC Priv positive rate: 0.92934 (0.04638)
FC Prot positive rate: 0.66188 (0.06089)


======= External Model Evaluation =======
Model: logreg (C=1.0)
Accuracy (val): 0.7560
Statistical Parity (prot - priv): -0.06587
======= 

In [126]:
fairness

array([-0.06586826, -0.06586826, -0.06387226, -0.06187625, -0.05988024,
       -0.05588822, -0.05788423, -0.05988024, -0.05988024, -0.05588822,
       -0.05988024])

In [127]:
fairness=[]
for i in range(0,11):
    budget = 2*i*X_datapool.shape[0]//100
    method = 'Moderate'
    

    st = System_T(
        (train_data, train_label),
        (val_data, val_label),
        val_data_dict,
        initial_data_array,
        num_class, num_label,
        slice_index,
        add_data_dict,
        favorable_label=1,
        privileged_slice_indices=priv_slices,
        protected_slice_indices=prot_slices
    )

    sp=st.selective_collect(
        budget=budget,
        k=10,
        batch_size=32,
        lr=lr,
        epochs=2000,
        cost_func=cost_func,
        Lambda=0.1,
        num_iter=5,
        slice_desc=slice_desc,
        strategy=method,
        show_figure=False
    )
    fairness.append(sp)



======= Collect Data =======
[0 0]
Total Cost: 1, Remaining Budget: -1.0

======= Performance (FC summary) =======
[0 0]
Number of iteration: 1
Strategy: Moderate, Lambda: 0.1, Initial Budget: 0
FC Loss: 0.91589 (0.34651)
FC Group Disparity (prot - priv): -0.22914 (0.11307)
FC Priv positive rate: 0.94132 (0.05028)
FC Prot positive rate: 0.71218 (0.06371)


======= External Model Evaluation =======
Model: logreg (C=1.0)
Accuracy (val): 0.7520
Statistical Parity (prot - priv): -0.06587
======= Collect Data =======
[605 599]
Total Cost: 1205, Remaining Budget: -1.0

======= Performance (FC summary) =======
[605 599]
Number of iteration: 1
Strategy: Moderate, Lambda: 0.1, Initial Budget: 1204
FC Loss: 0.81011 (0.13163)
FC Group Disparity (prot - priv): -0.21717 (0.06837)
FC Priv positive rate: 0.91697 (0.03714)
FC Prot positive rate: 0.69980 (0.06001)


======= External Model Evaluation =======
Model: logreg (C=1.0)
Accuracy (val): 0.7560
Statistical Parity (prot - priv): -0.06587
======= 

In [128]:
fairness

[-0.22914171656686627,
 -0.2171656686626747,
 -0.1772455089820359,
 -0.1528942115768463,
 -0.1540918163672655,
 -0.13812375249500997,
 -0.1497005988023952,
 -0.1441117764471058,
 -0.11696606786427144,
 -0.11497005988023953,
 -0.1129740518962076]

# Original ( with no data acquisition ) 

In [129]:
'''cost_func = [1] * num_class
lr = 0.001

ori = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, num_label, slice_index, add_data_dict, method='Uniform')
ori.performance(budget=0, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)'''

"cost_func = [1] * num_class\nlr = 0.001\n\nori = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, \n                initial_data_array, num_class, num_label, slice_index, add_data_dict, method='Uniform')\nori.performance(budget=0, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)"

# System T Demo on AdultCensus

In [130]:
budget = 300
cost_func = [1] * num_class
lr = 0.001

# Assuming slice_index order: 0=Male-White, 1=Male-Black, 2=Female-White, 3=Female-Black
male_slice_indices = [0, 1]

uni = Baseline(
    (train_data, train_label),
    (val_data, val_label),
    val_data_dict, 
    initial_data_array,
    num_class, num_label,
    slice_index, add_data_dict,
    method='Uniform',
    favorable_label=1,
    privileged_slice_indices=male_slice_indices
)

uni.performance(
    budget=budget,
    cost_func=cost_func,
    num_iter=10,
    batch_size=32,
    lr=lr,
    epochs=2000
)

Method: Uniform, Budget: 300
======= Collect Data =======
[150 150]
======= Performance (Group Disparity: prot - priv) =======
Loss: 0.78277 (0.04695)
Group Disparity (prot - priv): 0.00000 (0.00000)
Priv positive rate: 0.00000 (0.00000)
Prot positive rate: 0.00000 (0.00000)



## Use 300 budget, lambda=0.1, "Moderate" strategy

In [131]:
'''budget = 300
method = 'Moderate'

st = System_T((train_data, train_label), (val_data, val_label), val_data_dict, initial_data_array, num_class, num_label, slice_index, add_data_dict)
st.selective_collect(budget=budget, k=10, batch_size=32, lr = lr, epochs=2000, cost_func=cost_func, 
                 Lambda=0.1, num_iter=5, slice_desc=slice_desc, strategy=method, show_figure=False)'''

"budget = 300\nmethod = 'Moderate'\n\nst = System_T((train_data, train_label), (val_data, val_label), val_data_dict, initial_data_array, num_class, num_label, slice_index, add_data_dict)\nst.selective_collect(budget=budget, k=10, batch_size=32, lr = lr, epochs=2000, cost_func=cost_func, \n                 Lambda=0.1, num_iter=5, slice_desc=slice_desc, strategy=method, show_figure=False)"

# Baseline: Uniform ( = Water filling )

## For a basic setting, Uniform method is equivalent to Water filling method

In [132]:
'''budget = 300
uni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, num_label, slice_index, add_data_dict, method='Uniform')
uni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)'''

"budget = 300\nuni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, \n                initial_data_array, num_class, num_label, slice_index, add_data_dict, method='Uniform')\nuni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)"

# Summary of results

| Method | Loss | Avg.EER |
|:---------------------:|:---------------------:|:---------------------:|
| Original | 0.26328 (± 0.00101) | 0.10336 (± 0.00081) |
| Uniform | 0.25707 (± 0.00103) | 0.09862 (± 0.00043) |
| Water filling | 0.25707 (± 0.00103) | 0.09862 (± 0.00043) |
| Moderate (ours) | 0.25341 (± 0.00050) | 0.09448 (± 0.00026) |